Module System
---

Rust has a number of features that allow you to manage your code’s organization,  
including which details are exposed, which details are private, and what names  
are in each scope in your programs. These features, sometimes collectively  
referred to as the module system, include:  

- **`Packages`**: A Cargo feature that lets you build, test, and share crates
- **`Crates`**: A tree of modules that produces a library or executable
- **`Modules and use`**: Let you control the organization, scope, and privacy of paths
- **`Paths`**: A way of naming an item, such as a struct, function, or module

> In this chapter, we’ll cover all these features, discuss how they interact,  
and explain how to use them to manage scope. By the end, you should have a solid  
understanding of the module system and be able to work with scopes like a pro!

Packages and Crates
---

A crate is the smallest amount of code that the Rust compiler considers at a  
time. Even if you run rustc rather than cargo and pass a single source code file  
(as we did all the way back in the “Writing and Running a Rust Program” section  
of Chapter 1), the compiler considers that file to be a crate. Crates can  
contain modules, and the modules may be defined in other files that get compiled  
with the crate, as we’ll see in the coming sections.

A crate can come in one of two forms:  

- **`Binary crates`** are programs you can compile to an executable that you can run,  
such as a command-line program or a server. Each must have a function called  
main that defines what happens when the executable runs. All the crates we’ve  
created so far have been binary crates.
- **`Library crates`** don’t have a main function, and they don’t compile to an  
executable. Instead, they define functionality intended to be shared with  
multiple projects. For example, the rand crate we used in Chapter 2 provides  
functionality that generates random numbers. Most of the time when Rustaceans  
say “crate”, they mean library crate, and they use “crate” interchangeably with  
the general programming concept of a “library".